In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
chongqing = ee.FeatureCollection('users/311605001111/YangtzeCity/chongqing')
Map.addLayer(chongqing, {}, "chongqing")
Map.centerObject(chongqing,7)

In [4]:
# 去云掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)
# landsat8 图像可视化参数
visParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

In [5]:
def NDVI(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    return image.addBands(mndwi)

def EVI(image):
    evi = image.expression('((nir - red) / (nir + 6*red - 7.5*blue + 1)) * 2.5',
        {
          'blue': image.select('B2'),    # 0.452-0.512μm, blue
          'nir': image.select('B5'),    # 0.851-0.879μm, nir
          'red': image.select('B4'),    # 0.636-0.673μm, red
        }).rename('EVI')
    return image.addBands(evi)
# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def AWEIsh(image):
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)

# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# 分类标签
label = 'waterclass'

In [6]:
def area_after_clip(image):
    image_clip = image.clip(chongqing).select('B2').gt(0)
    image_area = image_clip.multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': chongqing.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    return image.clip(chongqing).set({'area': areas.get('B2')})

In [7]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(chongqing).select('constant').rename('waterclass')
# Map.addLayer(basemap,{'palette':['white']},"basemap")
basemap1 = ee.Image.constant(0).clip(chongqing).select('constant').rename('water_class')

# 训练&分类

In [8]:
landsat8_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(chongqing) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr)
print(landsat8_images.size().getInfo())
landsat8_images_filter = landsat8_images.map(area_after_clip).filter(ee.Filter.gt('area',0))
print(landsat8_images_filter.size().getInfo())

landsat8_map = landsat8_images_filter.map(NDVI).map(mNDWI).map(EVI).map(AWEIsh)
Map.addLayer(landsat8_map,visParams,'landsat image')
# 计算有效像元的个数
validPixel = landsat8_images.count().select('B2').clip(chongqing).rename('count')
# Map.addLayer(count_image,{},'count_image')

174
136


In [ ]:
def imageSample(image):
    chongqing_mNDWI_NDVI = image.select('mNDWI').gt(image.select('NDVI'))
    chongqing_waterextent = chongqing_mNDWI_NDVI.clip(chongqing).select('mNDWI').rename('waterclass')
    points = chongqing_waterextent.sample(**{
        'region': chongqing,
        'scale': 30,
        'numPixels': 120,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    # Overlay the points on the imagery to get training.
    sample_point = image.select(bands).sampleRegions(**{
        'collection': points,
        'properties': [label],
        'scale': 30
    })
    return sample_point

In [ ]:
points_collection = landsat8_map.map(imageSample).flatten()
print('total sample number:{}'.format(points_collection.size().getInfo()))

In [ ]:
# print(points_collection.first().getInfo())
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',0))
# print('no-water sample number:{}'.format(samplepoint_water.size().getInfo()))
samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
print('water sample number:{}'.format(samplepoint_water.size().getInfo()))
# print(samplepoint_water.first().getInfo())

trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
#对Landsat-8进行分类
def training(image):
    return image.clip(chongqing).select(bands).classify(trainedClassifier).eq(1).selfMask().select('classification').rename('waterclass')
landsatImage_classify = landsat8_map.map(training)
waterpixel_count = landsatImage_classify.sum()
# print(landsatImage_classify.size().getInfo())
# Map.addLayer(waterpixel_count,{},"waterPixelCount")
chongqing_waterfrequency = waterpixel_count.select('waterclass').divide(validPixel.select('count')).select('waterclass').rename('frequency')
chongqing_permanentwater = chongqing_waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')

Map.addLayer(chongqing_permanentwater,{'palette':['blue']},"chongqing_permanentwater")

In [ ]:
# 添加JRC数据，做对比
JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(chongqing)
JRC_permanentwater = JRC_wuhan_water.eq(3).select("waterClass").rename('jrcwaterclass')
Map.addLayer(JRC_permanentwater.selfMask(),{'palette':['cyan']},'jrcpermanent')

In [ ]:
imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
permanent_water_layer = imagecollection.sum()
Map.addLayer(permanent_water_layer,{'min':0,"max":1,'palette':['white','blue']},'waterpermanent')

Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(wuhan_ee).select('b1').gt(74).select('b1').rename('water_class')
wuhan_Maryland = ee.ImageCollection([Maryland_2018,basemap1]).sum()
Map.addLayer(wuhan_Maryland,{'min':0,"max":1,'palette':['white','red']},'wuhan_Maryland')

In [ ]:
# Map.addLayer(wuhan_permanentwater,{'palette':['yellow']},"permanentwater")

In [ ]:
# visualization = {
#     'bands': ['waterClass'],
#     'min': 0.0,
#   'max': 3.0,
#   'palette': ['cccccc', 'ffffff', '99d9ea', '0000ff']
# }
# JRC = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(wuhan_ee)
# Map.addLayer(JRC,visualization,'JRC')

In [ ]:
# # 添加JRC数据，做对比
# JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(wuhan_ee)
# JRC_permanentwater = JRC_wuhan_water.eq(3).select("waterClass").rename('jrcwaterclass')
# Map.addLayer(JRC_permanentwater.selfMask(),{'palette':['blue']},'jrcpermanent')

# 验证

## 使用JRC验证

In [ ]:
imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
# .select('frequency').rename('waterclass')
permanent_water_layer = imagecollection.sum()
# Map.addLayer(permanent_water_layer,{'min':0,"max":1,'palette':['white','blue']},'waterpermanent')

# 使用JRC做验证
JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(wuhan_ee)
JRC_permanentwater = JRC_wuhan_water.eq(3).select("waterClass").rename('jrcwaterclass')
JRC_samplepoints = JRC_permanentwater.sample(**{
    'region': wuhan_ee,
    'scale': 30,
    'numPixels': 3000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('JRC total number of sample point:{}'.format(JRC_samplepoints.size().getInfo()))
# print(JRC_samplepoints.first().getInfo())
waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))
# Overlay the points on the imagery to get training.
JRC_sample_point = permanent_water_layer.sampleRegions(**{
    'collection': JRC_samplepoints,
    'properties': ['jrcwaterclass'],
    'scale': 30
})
print(JRC_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = JRC_sample_point.errorMatrix('jrcwaterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total caccuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

In [ ]:
imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
# .select('frequency').rename('waterclass')
permanent_water_layer = imagecollection.sum()
# Map.addLayer(permanent_water_layer,{'min':0,"max":1,'palette':['white','blue']},'waterpermanent')

# 使用Maryland做验证

Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(wuhan_ee).select('b1').gt(74).select('b1').rename('water_class')
wuhan_Maryland = ee.ImageCollection([Maryland_2018,basemap1]).sum()
Maryland_samplepoints =wuhan_Maryland .sample(**{
    'region': wuhan_ee,
    'scale': 30,
    'numPixels': 3000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('Maryland total number of sample point:{}'.format(Maryland_samplepoints.size().getInfo()))
# print(Maryland_samplepoints.first().getInfo())
waterclass = Maryland_samplepoints.filter(ee.Filter.eq('water_class',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))
# Overlay the points on the imagery to get training.
Maryland_sample_point = permanent_water_layer.sampleRegions(**{
    'collection': Maryland_samplepoints,
    'properties': ['water_class'],
    'scale': 30
})
print(Maryland_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = Maryland_sample_point.errorMatrix('water_class', 'waterclass')
print(test_accuracy.getInfo())
print('total caccuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

# 水体面积

In [ ]:
# calculate the area of permanent water
waterarea = wuhan_permanentwater.multiply(ee.Image.pixelArea()).divide(1e6)
areas = waterarea.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': wuhan_ee.geometry(),
    'scale': 30,
    'maxPixels': 1e14
})
print(areas.getInfo())

In [ ]:
# from matplotlib import pyplot as plt
# import numpy as np
# import matplotlib
# from geemap import cartoee

In [ ]:
# from geemap import cartoee
# region = [113.6, 29.9, 115.1, 31.4] 
# vis = {'min':0, 'max':1}
# fig = plt.figure(figsize=(12, 8))
# cmap = 'Blues'
# # use cartoee to get a map
# ax = cartoee.get_map(wuhan_waterfrequency, region=region, vis_params=vis,cmap = cmap)
# # add a colorbar to the map using the visualization params we passed to the map
# cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
# # ticks=range(0,50,10)
# # add gridlines to the map at a specified interval
# cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
# ax.set_title(label = 'waterbody frequency in 2018(MNDWI>NDVI)', fontsize=20)

In [ ]:
# from geemap import cartoee
# region = [113.6, 29.9, 115.1, 31.4] 
# vis = {'bands': ['waterclass'], 'palette': ['blue'], 'min': 0.0, 'max': 1.0, 'opacity': 1.0}
# fig = plt.figure(figsize=(12, 8))
# # use cartoee to get a map
# ax = cartoee.get_map(wuhan_permanentwater, region=region, vis_params=vis)
# # add gridlines to the map at a specified interval
# cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
# ax.set_title(label = 'permanent surface water in 2018(MNDWI>NDVI)', fontsize=20)